In [1]:
from sympy import *

In [2]:
q_n12, q_n1, q_n, a, b, h, M = symbols("q^{(n+1/2)} q^{(n+1)} q^{(n)} a b h M")
f_n, f_n1, eps, eps_, theta_n, theta_n1 = symbols(r"f^{(n)} f^{(n+1)} \epsilon \epsilon' \theta^{(n)} \theta^{(n+1)}")
alpha, C, T = symbols(r"\alpha C T")

In [3]:
vel1 = sqrt(a)*q_n + Rational(1, 2)*C*f_n + alpha*eps
vel1

C*f^{(n)}/2 + \alpha*\epsilon + sqrt(a)*q^{(n)}

In [4]:
pos1 = theta_n + C*q_n12
pos1

C*q^{(n+1/2)} + \theta^{(n)}

In [5]:
vel2 = sqrt(a)*q_n12 + Rational(1, 2)*sqrt(a)*C*f_n1 + alpha*eps_
vel2

C*sqrt(a)*f^{(n+1)}/2 + \alpha*\epsilon' + sqrt(a)*q^{(n+1/2)}

In [6]:
C_def =  b*h/sqrt(M)
alpha_def = sqrt((1-a)*T)

In [7]:
vel1_expanded = vel1.subs(C, C_def).subs(alpha, alpha_def)
vel1_expanded

\epsilon*sqrt(T*(1 - a)) + sqrt(a)*q^{(n)} + b*f^{(n)}*h/(2*sqrt(M))

In [8]:
pos1.subs(C, C_def)

\theta^{(n)} + b*h*q^{(n+1/2)}/sqrt(M)

In [9]:
vel2_expanded = vel2.subs(C, C_def).subs(alpha, alpha_def)
vel2_expanded

\epsilon'*sqrt(T*(1 - a)) + sqrt(a)*q^{(n+1/2)} + sqrt(a)*b*f^{(n+1)}*h/(2*sqrt(M))

In [10]:
expand(simplify(vel1_expanded.subs(q_n, vel2_expanded).subs(f_n, f_n1)))

\epsilon*sqrt(-T*a + T) + \epsilon'*sqrt(a)*sqrt(-T*a + T) + a*q^{(n+1/2)} + a*b*f^{(n+1)}*h/(2*sqrt(M)) + b*f^{(n+1)}*h/(2*sqrt(M))

In [11]:
solution = linsolve([
    Eq(theta_n1, pos1.subs(q_n12, vel1)),
    Eq(q_n1, vel2.subs(q_n12, vel1))],
    [eps, eps_])

In [12]:
eps_def, eps__def = next(iter(solution))
eps_def

(-C**2*f^{(n)}/2 - C*sqrt(a)*q^{(n)} - \theta^{(n)} + \theta^{(n+1)})/(C*\alpha)

In [23]:
c_2 = symbols("c_2")
alpha_def_c_2 = c_2 / sqrt(M)

eps_def_alt = expand(eps_def).subs(C*alpha, simplify(alpha_def_c_2 * C_def))\
                        .subs(alpha, alpha_def_c_2).subs(C, C_def)
eps_def_alt

-sqrt(M)*sqrt(a)*q^{(n)}/c_2 - M*\theta^{(n)}/(b*c_2*h) + M*\theta^{(n+1)}/(b*c_2*h) - b*f^{(n)}*h/(2*c_2)

In [24]:
eps__def_alt = -expand(eps__def).subs(C*alpha, simplify(alpha_def_c_2 * C_def))\
                        .subs(alpha, alpha_def_c_2).subs(C, C_def)
eps__def_alt

-sqrt(M)*q^{(n+1)}/c_2 - M*\theta^{(n)}*sqrt(a)/(b*c_2*h) + M*\theta^{(n+1)}*sqrt(a)/(b*c_2*h) + sqrt(a)*b*f^{(n+1)}*h/(2*c_2)

In [15]:
log_density1 = -Rational(1, 2)*(eps_def_alt**2 + eps__def_alt**2)
log_density1

-(-sqrt(M)*q^{(n+1)}/c_2 - M*\theta^{(n)}*sqrt(a)/(b*c_2*h) + M*\theta^{(n+1)}*sqrt(a)/(b*c_2*h) + sqrt(a)*b*f^{(n+1)}*h/(2*c_2))**2/2 - (-sqrt(M)*sqrt(a)*q^{(n)}/c_2 - M*\theta^{(n)}/(b*c_2*h) + M*\theta^{(n+1)}/(b*c_2*h) - b*f^{(n)}*h/(2*c_2))**2/2

In [16]:
tmp = symbols('\text{tmp}')
log_density2 = (log_density1
                .subs(f_n, tmp).subs(f_n1, f_n).subs(tmp, f_n1)
                .subs(theta_n, tmp).subs(theta_n1, theta_n).subs(tmp, theta_n1)
                .subs(q_n, tmp).subs(q_n1, -q_n).subs(tmp, -q_n1))
log_density2

-(sqrt(M)*q^{(n)}/c_2 + M*\theta^{(n)}*sqrt(a)/(b*c_2*h) - M*\theta^{(n+1)}*sqrt(a)/(b*c_2*h) + sqrt(a)*b*f^{(n)}*h/(2*c_2))**2/2 - (sqrt(M)*sqrt(a)*q^{(n+1)}/c_2 + M*\theta^{(n)}/(b*c_2*h) - M*\theta^{(n+1)}/(b*c_2*h) - b*f^{(n+1)}*h/(2*c_2))**2/2

In [17]:
x = simplify(simplify((log_density1 - log_density2)*T).subs(c_2**2, (1-a)*M*T))

In [18]:
x

-\theta^{(n)}*f^{(n)}/2 - \theta^{(n)}*f^{(n+1)}/2 + \theta^{(n+1)}*f^{(n)}/2 + \theta^{(n+1)}*f^{(n+1)}/2 + q^{(n)}**2/2 - q^{(n+1)}**2/2 - b**2*f^{(n)}**2*h**2/(8*M) + b**2*f^{(n+1)}**2*h**2/(8*M)

# SGLD version

In [64]:
ld1_sgld = log_density1.subs(a, 0)
ld1_sgld

-M*q^{(n+1)}**2/(2*c_2**2) - (-M*\theta^{(n)}/(b*c_2*h) + M*\theta^{(n+1)}/(b*c_2*h) - b*f^{(n)}*h/(2*c_2))**2/2

In [65]:
ld2_sgld = log_density2.subs(a, 0)
ld2_sgld

-M*q^{(n)}**2/(2*c_2**2) - (M*\theta^{(n)}/(b*c_2*h) - M*\theta^{(n+1)}/(b*c_2*h) - b*f^{(n+1)}*h/(2*c_2))**2/2

In [67]:
x = expand(expand((ld1_sgld - ld2_sgld)*T).subs(c_2**2, M*T))
x

-\theta^{(n)}*f^{(n)}/2 - \theta^{(n)}*f^{(n+1)}/2 + \theta^{(n+1)}*f^{(n)}/2 + \theta^{(n+1)}*f^{(n+1)}/2 + q^{(n)}**2/2 - q^{(n+1)}**2/2 - b**2*f^{(n)}**2*h**2/(8*M) + b**2*f^{(n+1)}**2*h**2/(8*M)

In [57]:
ld1_sgld = -Rational(1, 4)/h**2 * (theta_n1 - theta_n - h**2 * f_n)**2
ld1_sgld

-(-\theta^{(n)} + \theta^{(n+1)} - f^{(n)}*h**2)**2/(4*h**2)

In [58]:
ld2_sgld = (ld1_sgld
                .subs(f_n, tmp).subs(f_n1, f_n).subs(tmp, f_n1)
                .subs(theta_n, tmp).subs(theta_n1, theta_n).subs(tmp, theta_n1)
                .subs(q_n, tmp).subs(q_n1, -q_n).subs(tmp, -q_n1))
ld2_sgld

-(\theta^{(n)} - \theta^{(n+1)} - f^{(n+1)}*h**2)**2/(4*h**2)

In [60]:
x = simplify(expand((ld1_sgld - ld2_sgld)).subs(c_2**2, M*T))
x

-\theta^{(n)}*f^{(n)}/2 - \theta^{(n)}*f^{(n+1)}/2 + \theta^{(n+1)}*f^{(n)}/2 + \theta^{(n+1)}*f^{(n+1)}/2 - f^{(n)}**2*h**2/4 + f^{(n+1)}**2*h**2/4